In [ ]:
from pathlib import Path

from hloc import (
    extract_features,
    match_dense,
    match_features,
    pairs_from_retrieval,
    reconstruction,
    visualization,
)
from hloc.utils import viz_3d
import shutil
import time 


In [ ]:

images = Path("takeout-1-001")

shutil.rmtree("outputs/sfm", ignore_errors=True)
outputs = Path("outputs/sfm/")
sfm_pairs = outputs / "pairs-netvlad.txt"
sfm_dir = outputs / "sfm_superpoint+lightglue"

retrieval_conf = extract_features.confs["netvlad"]
feature_conf = extract_features.confs["superpoint_aachen"]
matcher_conf = match_features.confs["superpoint+lightglue"]
# matcher_conf = match_dense.confs["loftr_aachen"]

time1 = time.time()

In [ ]:
retrieval_path = extract_features.main(retrieval_conf, images, outputs)
pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=5)

In [ ]:
feature_path = extract_features.main(feature_conf, images, outputs)


In [ ]:
match_path = match_features.main(
    matcher_conf, sfm_pairs, feature_conf["output"], outputs
)

In [ ]:
model = reconstruction.main(sfm_dir, images, sfm_pairs, feature_path, match_path)

In [ ]:
time2 = time.time()
print(f"Total time taken: {time2 - time1} seconds")

In [ ]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
fig.show()

# Test the api

In [ ]:
import os
import time
import requests

def post_images_to_api(images_dir, api_url="http://localhost:8000/sfm", multiply=1, output_zip_path="output.zip"):
    files = []
    image_files = [
        f
        for f in os.listdir(images_dir)
        if os.path.isfile(os.path.join(images_dir, f))
        and f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]
    if not image_files:
        print("No images found in the directory.")
        return

    for i in range(multiply):
        for filename in image_files:
            filepath = os.path.join(images_dir, filename)
            send_name = (
                f"{os.path.splitext(filename)[0]}_copy{i}{os.path.splitext(filename)[1]}"
                if multiply > 1
                else filename
            )
            files.append(("images", (send_name, open(filepath, "rb"), "image/jpeg")))

    print("Sending images...")
    with requests.post(api_url, files=files, stream=True) as response:
        print(f"Response status code: {response.status_code}")
        if response.status_code == 200:
            with open(output_zip_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            print(f"Zip file saved to {output_zip_path}")
        else:
            print("Error in API response:", response.text)


time1 = time.time()
post_images_to_api(
    "takeout-1-001",
    api_url="http://localhost:8000/sfm",
    multiply=1,
    output_zip_path="sfm_output.zip",
)
print("API call completed.")
time2 = time.time()
print(f"API call time taken: {time2 - time1:.2f} seconds")


Sending images...
